This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [10]:
# # !pip install pydantic>=1.10.12
# # !pip install pytest>=7.4.0
# # !pip install sentence-transformers>=2.2.0
# # !pip install transformers>=4.31.0
# # !pip install chromadb==0.4.18
# # !pip install --upgrade --quiet  langchain-openai
# # !pip install --upgrade openai
# !pip install langchain==0.0.316 --user
# !pip install openai==0.28.1 --user
# # !pip install -U langchain chromadb
# # !pip install chromadb==0.4.15
# # !pip install openai==1.12.0
# !pip install langchain==0.0.316
# !pip install openai==0.28.1
# # !pip install langchain-openai --user
# # !pip install langchain-community langchain-core
# # !pip install --upgrade langchain
# # !pip install --upgrade pip --user
# # !pip install langchain --upgrade --user
# # !pip install --upgrade openai --user

In [3]:
import io
import os
import sys
import shutil
# Import Azure OpenAI
import openai
import numpy as np
import pandas as pd
import io

from langchain.prompts import PromptTemplate
from chromadb.utils import embedding_functions
from chromadb import Documents, EmbeddingFunction, Embeddings
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_openai import AzureOpenAI, AzureOpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import ChatPromptTemplate
from langchain.evaluation import load_evaluator
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma
from dataclasses import dataclass
from langchain.chat_models import ChatOpenAI
from langchain_openai import AzureOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain_openai import AzureChatOpenAI

### Generating Real Estate Listings with an LLM
- using a Large Language Model (LLM) to generate at least 10 diverse and realistic real estate listings containing facts about the real estate.

In [18]:

prompt='''
Create a comprehensive CSV file encapsulating the essence of {num_listings} distinct real estate listings. Each listing should be organized into columns, showcasing the following attributes:

- Neighborhood: Identify the neighborhood where the property resides, such as "Green Oaks."
- Price: State the property's market price in USD, formatted (e.g., "$800,000").
- Bedrooms: Enumerate the bedrooms within the property (e.g., 3).
- Bathrooms: Count the property's bathrooms (e.g., 2).
- House Size: Detail the property's square footage (e.g., "2,000 sqft").

Elevate each property's presentation with a comprehensive paragraph that accentuates its unique features, amenities, and sustainable qualities. Highlight elements like energy-efficient appliances, solar panel integration, use of sustainable materials, and the presence of gardens.

Example Entry Format:
Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description
Green Oaks,"$800,000",3,2,"2,000 sqft","Nestled in Green Oaks, this eco-friendly haven features a 3-bedroom, 2-bathroom layout with solar panels and efficient insulation. Highlights include abundant natural light, hardwood floors, and an open-concept kitchen that leads to a lush backyard, embodying a sanctuary for eco-conscious living. The neighborhood of Green Oaks is celebrated for its vibrant and environmentally-aware community, boasting organic stores, community gardens, and convenient transit options, rendering it perfect for those prioritizing sustainability and community engagement."


Ensure the description not only reflects the property's allure, such as its eco-friendly design and comfortable living spaces but also paints a vivid picture of the neighborhood's character. Emphasize community elements like Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.

Structure the CSV with clear headers for each column. Follow the example provided to format each subsequent row with information specific to a different property listing.
Make sure you generate {num_listings} distinct and unique listings.
'''

openai.api_type = "azure"
openai.api_version = "2023-03-15-preview" 
openai.api_base =  "xxx"  # Your Azure OpenAI resource's endpoint value.
openai.api_key = "xxx"

gen_listing_prompt = PromptTemplate.from_template(prompt).format(num_listings=15)

conversation=[{"role": "system", "content": f"{gen_listing_prompt}"}]

response = openai.ChatCompletion.create(
    engine="Enigma-code", # The deployment name you chose when you deployed the GPT-3.5-Turbo or GPT-4 model.
    messages = conversation,
    temperature=0,
    max_tokens=2400
)
agent_response = response.choices[0].message.content
conversation.append({"role": "assistant", "content": agent_response})
print(agent_response)



Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description
Westwood,"$1,200,000",4,3,"2,500 sqft","This stunning 4-bedroom, 3-bathroom home in Westwood boasts a spacious open floor plan, perfect for entertaining. The property features a chef's kitchen with stainless steel appliances, a large master suite with a walk-in closet, and a backyard oasis with a pool and spa. The neighborhood of Westwood is known for its excellent schools, tree-lined streets, and proximity to trendy restaurants and shops."
Brentwood,"$2,500,000",5,4,"3,800 sqft","Nestled in the heart of Brentwood, this luxurious 5-bedroom, 4-bathroom home boasts high-end finishes and an open-concept floor plan. The property features a gourmet kitchen with top-of-the-line appliances, a spacious master suite with a fireplace and balcony, and a backyard with a built-in BBQ and fire pit. The neighborhood of Brentwood is known for its upscale shops and restaurants, excellent schools, and easy access to the beach."
Santa Monica,"$

In [19]:
type(agent_response)

str

In [22]:
print("Datatype of the agent response: ", type(agent_response), "\n")
# Transform our listings to dataframe
homes_df = pd.read_csv(io.StringIO(agent_response), sep=",")
display(homes_df)

# Saving the Homes listings in a csv file
homes_df.to_csv('Homes_listings.csv', index=None)

Datatype of the agent response:  <class 'str'> 



,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description
0,Westwood,"$1,200,000",4,3,"2,500 sqft","This stunning 4-bedroom, 3-bathroom home in We..."
1,Brentwood,"$2,500,000",5,4,"3,800 sqft","Nestled in the heart of Brentwood, this luxuri..."
2,Santa Monica,"$1,800,000",3,2,"1,800 sqft","This charming 3-bedroom, 2-bathroom home in Sa..."
3,Venice,"$1,500,000",2,2,"1,500 sqft","Located in the heart of Venice, this stunning ..."
4,Marina Del Rey,"$1,700,000",3,3,"2,200 sqft","This beautiful 3-bedroom, 3-bathroom home in M..."
5,Culver City,"$1,000,000",2,2,"1,200 sqft","This charming 2-bedroom, 2-bathroom home in Cu..."
6,Playa Vista,"$1,600,000",4,3,"2,500 sqft","Located in the heart of Playa Vista, this stun..."
7,Westchester,"$900,000",3,2,"1,500 sqft","This charming 3-bedroom, 2-bathroom home in We..."
8,Pacific Palisades,"$3,000,000",5,4,"4,000 sqft","Nestled in the hills of Pacific Palisades, thi..."
9,Beverly Hills,"$5,000,000",6,5,"5,500 sqft","This stunning 6-bedroom, 5-bathroom home in Be..."


### Creating a Vector database and Storing Listings

In [12]:
import pandas as pd

homes_df = pd.read_csv('Homes_listings.csv')
homes_df.head()

,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description
0,Westwood,"$1,200,000",4,3,"2,500 sqft","This stunning 4-bedroom, 3-bathroom home in We..."
1,Brentwood,"$2,500,000",5,4,"3,800 sqft","Nestled in the heart of Brentwood, this luxuri..."
2,Santa Monica,"$1,800,000",3,2,"1,800 sqft","This charming 3-bedroom, 2-bathroom home in Sa..."
3,Venice,"$1,500,000",2,2,"1,500 sqft","Located in the heart of Venice, this stunning ..."
4,Marina Del Rey,"$1,700,000",3,3,"2,200 sqft","This beautiful 3-bedroom, 3-bathroom home in M..."


In [13]:



docs = []
for index, row in homes_df.iterrows():
    docs.append(Document(page_content=row['Description'], metadata={'id': str(index)}))
    
# Split Text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=100,
    length_function=len,
    add_start_index=True,
)
chunks = text_splitter.split_documents(docs)
print(f"Split {len(docs)} documents into {len(chunks)} chunks.")

os.environ["OPENAI_API_VERSION"] = '2023-03-15-preview'
os.environ["AZURE_OPENAI_ENDPOINT"] = "xxx"
os.environ["AZURE_OPENAI_API_KEY"] = "xxx"



if chunks:
    document = chunks[10]
    print(document.page_content)
    print(document.metadata)
    
import openai

openai.api_type = "azure"
openai.api_key = "xxx"
openai.api_base = "xxx"
openai.api_version = "2023-03-15-preview"

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="Pandoras-Box",
    openai_api_version="2023-03-15-preview",
)
# print(embeddings)
  
    
db = Chroma.from_documents(chunks, embeddings , persist_directory= "chroma")
db.persist()

Split 15 documents into 55 chunks.
Located in the heart of Venice, this stunning 2-bedroom, 2-bathroom home features an open-concept living space with high ceilings and skylights, a gourmet kitchen with stainless steel appliances, and
{'id': '3', 'start_index': 0}


### Semantic Search of Listings Based on Buyer Preferences
- listings are semantically searched based on given buyer preferences. The search should return listings that closely match the input preferences.

In [15]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain

from langchain_openai import AzureOpenAI
from langchain_openai import AzureOpenAIEmbeddings


import os
#text-embedding-ada-002
#Pandoras-Box
os.environ["OPENAI_API_VERSION"] = '2023-03-15-preview'
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://enigma-code.openai.azure.com"
os.environ["AZURE_OPENAI_API_KEY"] = "ea5e645a87f04a72beed5bdffde87da2"

loader = CSVLoader(file_path='Homes_listings.csv')
docs = loader.load()

# print(docs)
llm = AzureOpenAI(deployment_name='Enigma-code', temperature=0, max_tokens = 2000)

splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(docs)

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="Pandoras-Box",
    openai_api_version="2023-03-15-preview",
)


In [16]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory
from langchain.chains import ConversationChain
from typing import Any, Dict, Optional, Tuple

In [18]:
llm = AzureOpenAI(deployment_name='Enigma-code', temperature=0, max_tokens = 1000)


personal_questions = [   
                        "How big do you want your house to be?",
                        "What are 3 most important things for you in choosing this property?", 
                        "Which amenities would you like?", 
                        "Which transportation options are important to you?",
                        "How urban do you want your neighborhood to be?",
]

#personal_answers = [ ] 
#for question in personal_questions:
#    answer = input(question)
#    personal_answers.append(answer)

personal_answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]
print(personal_answers)

history = ChatMessageHistory()
history.add_user_message(f"""You are AI Real Estate Agent. Ask user {len(personal_questions)} questions""")
for i in range(len(personal_questions)):
    history.add_ai_message(personal_questions[i])
    history.add_user_message(personal_answers[i])
    
summary_memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="Real_Estate_Agent_chat", 
    input_key="input",
    buffer=f"The human answered {len(personal_questions)} personal questions).",
    return_messages=True)

# you could choose to store some of the q/a in memory as well, in addition to original questions
class MementoBufferMemory(ConversationBufferMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        input_str, output_str = self._get_input_output(inputs, outputs)
        self.chat_memory.add_ai_message(output_str)
    
conversational_memory = MementoBufferMemory(
    chat_memory=history,
    memory_key="questions_and_answers", 
    input_key="input"
)

# Combined
memory = CombinedMemory(memories=[conversational_memory, summary_memory])
RECOMMENDER_TEMPLATE = """The following is a friendly conversation between a human and an AI Real Estate Agent. 
                        The AI is follows human instructions and provides a home for a human based on the human's persona derived from their answers to questions. 

Previous conversation:
{Real_Estate_Agent_chat}

Personal Questions and Answers:
{questions_and_answers}
Human: {input}
Real Estate Agent:"""
PROMPT = PromptTemplate(
    input_variables=["Real_Estate_Agent_chat", "input", "questions_and_answers"],
    template=RECOMMENDER_TEMPLATE
)
recommender = ConversationChain(llm=llm, verbose=True, memory=memory, prompt=PROMPT)
 


final_recommendation = """Combine all the anwers in one big sentence"""
prediction = recommender.predict(input=final_recommendation)

query = """
Answer the question based only on the following context:

{context}

---

Given the context provided above, craft a response that not only answers the question , but also ensures that your explanation is distinct, captivating, and customized to align with the specified preferences. Strive to present your insights in a manner that resonates with the audience's interests and requirements
"""

use_chain_helper = False
if use_chain_helper:
    rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
    print(rag.run(query))
else:
    similar_docs = db.similarity_search(query, k=2)
    prompt = PromptTemplate.from_template(query).format(context=prediction)
    prompt = PromptTemplate(
        template="{query}\nContext: {context}",
        input_variables=["query", "context"],
    )
    chain = load_qa_chain(llm, prompt = prompt, chain_type="stuff")
    print(chain.run(input_documents=similar_docs, query = query))

['A comfortable three-bedroom house with a spacious kitchen and a cozy living room.', 'A quiet neighborhood, good local schools, and convenient shopping options.', 'A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.', 'Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.', 'A balance between suburban tranquility and access to urban amenities like restaurants and theaters.']


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI Real Estate Agent. 
                        The AI is follows human instructions and provides a home for a human based on the human's persona derived from their answers to questions. 

Previous conversation:
[SystemMessage(content='The human answered 5 personal questions).')]

Personal Questions and Answers:
Human: You are AI Real Estate Agent. Ask user 5 questions
AI: How big do you want your house to be?

C:\Users\tareka3\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 and a bustling nightlife. The property is located within walking distance of the Staples Center, LA Live, and the Convention Center, making it an ideal location for those who want to be in the heart of the action.

If you're looking for a luxurious and convenient place to stay in Downtown LA, this master suite is the perfect choice. With its spacious layout, walk-in closet, and rooftop deck, you'll have plenty of room to relax and unwind after a long day of exploring the city. And with its prime location, you'll be just steps away from some of the best restaurants, shops, and entertainment venues in the area. Whether you're in town for business or pleasure, this master suite is sure to exceed your expectations.  So, if you want to experience the best that Downtown LA has to offer, book your stay today!<|im_end|>
